In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys
import datetime as dt
import mp_utils as mp

from sklearn.pipeline import Pipeline

# used for train/test splits and cross validation
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

# used to impute mean for data and standardize for computational stability
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# logistic regression is our favourite model ever
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV # l2 regularized regression
from sklearn.linear_model import LassoCV

# used to calculate AUROC/accuracy
from sklearn import metrics

# used to create confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score

# gradient boosting - must download package https://github.com/dmlc/xgboost
import xgboost as xgb

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']

from __future__ import print_function 

# extract data from *all* patients to create init design matrix

In [ ]:
# query path
qpath = '/home/alistairewj/mimic-private-code/mortality-prediction/views-single-patient/'

# below config used on pc70
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name + ';')

files = ['bloodgasarterial','gcs','height','labs',
         'rass','rrt','rrt_range','uo','vasopressor','vent',
         'vitals','weight']

starttime = dt.datetime.now()
currenttime = starttime
print('Starting query extract - {}'.format(starttime))
data = dict()
for f in files:
    query = 'select * from mpap_' + f #+ ' where icustay_id in (200003,200024)'
    
    # replace variable in python because using variables in SQL w/o psql is ridiculously complicated
    data[f] = pd.read_sql_query(query,con)
    
    print('Finished {} - time elapsed: {}'.format(f, dt.datetime.now() - currenttime))
    currenttime = dt.datetime.now()

# extract static vars into a separate dataframe
df_static = pd.read_sql_query('select * from mpap_static_vars',con)
for dtvar in ['intime','outtime','deathtime']:
    df_static[dtvar] = pd.to_datetime(df_static[dtvar])

print('Finished all queries - time elapsed: {}'.format(dt.datetime.now() - starttime))
cur.close()
con.close()

# change charttime into timeelapsed

It's easier to work with seconds since admission, rather than an absolute charttime.

In [ ]:
# convert charttime to charttime_elapsed, starttime to starttime_elapsed, endtime to endtime_elapsed
df_static_add = df_static[['icustay_id','intime']].set_index('icustay_id')


for i, f in enumerate(data):
    # convert charttime / starttime / endtime into elapsed times
    # add in the intime to the dataframe
    data[f] = data[f].merge(df_static_add,how='left',left_on='icustay_id',right_index=True)
    
    
    # drop any missing ICUSTAY_ID or CHARTTIME from the frame
    idxRem = data[f]['icustay_id'].isnull()
    if np.sum(idxRem) > 0:
        print('{:20s}... removing {} rows with null ICUSTAY_ID.'.format(f, np.sum(idxRem)))
        data[f].drop(data[f].index[idxRem], axis=0, inplace=True)
    
    data[f]['icustay_id'] = data[f]['icustay_id'].astype(int)
        
        
    if 'charttime' in data[f].columns:
        idxRem = data[f]['charttime'].isnull()
        if np.sum(idxRem) > 0:
            print('{:20s}... removing {} rows with null CHARTTIME.'.format(f, np.sum(idxRem)))
            data[f].drop(data[f].index[idxRem], axis=0, inplace=True)
    
        # convert charttime to timestamp if it was not recognized because of missing data
        data[f]['charttime'] = pd.to_datetime(data[f]['charttime'])
            
            
    if 'starttime' in data[f].columns:
        if data[f].shape[0] == 0:
            # empty array
            data[f]['starttime_elapsed'] = np.zeros([0,])
            data[f]['endtime_elapsed'] = np.zeros([0,])
        else:
            data[f]['starttime_elapsed'] = (data[f]['starttime'] - data[f]['intime']) / np.timedelta64(1, 's')
            data[f]['endtime_elapsed'] = (data[f]['endtime'] - data[f]['intime']) / np.timedelta64(1, 's')
        
        data[f].drop('starttime', axis=1, inplace=True)
        data[f].drop('endtime', axis=1, inplace=True)
        
    elif 'charttime' in data[f].columns:
        if data[f].shape[0] == 0:
            # empty array
            data[f]['charttime_elapsed'] = np.zeros([0,])
        else:
            data[f]['charttime_elapsed'] =  (data[f]['charttime'] - data[f]['intime']) / np.timedelta64(1, 's')
        
        data[f].drop('charttime', axis=1, inplace=True)
    
    data[f].drop('intime', axis=1, inplace=True)

In [ ]:
df = mp.collapse_data(data)

In [ ]:
# now add in data where we have starttime/endtime, not just charttime


# dictionary mapping table names to column name of interest
colNameMap = {'vent': 'vent',
              'vasopressor': 'vasopressor',
              'rrt_range': 'rrt'}
rangeTbl = ['vent','vasopressor','rrt_range']


# initialize a dataframe with every possible time
charttime = df[['icustay_id','charttime_elapsed']].copy()

for f in rangeTbl:
    df_tmp = data[f]
    
    colName = colNameMap[f]
    
    # initialize the data to all zeros
    charttime.loc[:,colName] = np.zeros(charttime.shape[0])
    
    if df_tmp.shape[0] == 0:
        # there is no data for this table - continue
        continue
    
    # add in the values
    df_tmp.loc[:,colName + '_start'] = np.ones(df_tmp.shape[0])
    df_tmp.loc[:,colName + '_end'] = -1*np.ones(df_tmp.shape[0])
    
    # add the starttime/endtime as charttimes
    # this ensures that we have a row in the master dataframe corresponding to the start/end time of these observations
    charttime = pd.concat([charttime,
                           df_tmp[['icustay_id','starttime_elapsed', colName + '_start']]
                           .rename(columns = {'starttime_elapsed': 'charttime_elapsed', colName + '_start': colName})],
                          ignore_index=True)
    charttime = pd.concat( [charttime,
                            df_tmp[['icustay_id','endtime_elapsed', colName + '_end']]
                            .rename(columns = {'endtime_elapsed': 'charttime_elapsed', colName + '_end': colName})],
                         ignore_index=True)

charttime = charttime.drop_duplicates()

# set indices for data
df.set_index(['icustay_id','charttime_elapsed'],inplace=True)
charttime.set_index(['icustay_id','charttime_elapsed'],inplace=True)

In [ ]:
# create a variable which is 1 if the charttime is between starttime/endtime
for f in rangeTbl:
        
    # set the value of each column @ the respective starttime to 1
    colName = colNameMap[f]
    
    # impute 0 for NaNs now - this allows us to *subtract* 1 from the data
    # when done in this order, we prevent errors in cumsum if starttime == endtime
    charttime[colName].fillna(0,inplace=True)
    
# sort the data by the index
charttime.sort_index(axis=0, ascending=True, inplace=True)

for f in rangeTbl:
    colName = colNameMap[f]
    # cumulative sum each value
    # this will assign all charttimes after starttime to 1
    # the endtime will end up with a value of 0
    charttime.loc[:,colName] = charttime.loc[:,colName].cumsum(axis=0, dtype=int, skipna=True)

In [ ]:
# add in the charttime data to our full data
# note that we set the values explicitly to avoid
df = df.merge(charttime,
                left_index=True, right_index=True,
                suffixes=('','_range_variable'),
                how='outer')

# if the column already existed, a new column has been created, with suffix '_range_variable'
# loop through these vars, and merge them with the original var
# since they are binary flags, we can happily use max() to merge the data
colFix = [x for x in df.columns if '_range_variable' in x]
if len(colFix) > 0:
    for f in colFix:
        f_orig = f[:-15] # remove suffix

        df.loc[:,f_orig] = df[[f_orig, f]].apply(max, axis=1)
        df.drop(f,axis=1,inplace=True)
    
print(df.shape)

In [ ]:
# write the individual patient's data to file
# this takes >30 minutes
# each file is around 20-50kb
for iid in df.index.levels[0]:
    df_curr = df.loc[iid,:]
    df_curr.to_csv('./data/' + str(int(iid)) + '.csv')

Our dataframe is now prepared. It contains timestamped observations for a number of features.

The dataframe is completely denormalized, and is a bit inefficient space wise!

Next, we extract all the design matrices we would like:
    
    * using a random offset, 4 hour window, somewhere between `INTIME` and (`OUTTIME - 4 hours`)
    * using a random offset, 4 hour window, with offset for deaths fixed at DEATHTIME -  4 Hr
    * using a random offset, 4 hour window, with offset for deaths fixed at DEATHTIME -  8 Hr
    * using a random offset, 4 hour window, with offset for deaths fixed at DEATHTIME - 16 Hr
    * using a random offset, 4 hour window, with offset for deaths fixed at DEATHTIME - 24 Hr
    * using a fixed offset of hour 0, 4 hour window

This involves:

1. Setting a random seed
2. Defining the start_dict dictionary (start times for each iid)
3. Extracting/writing out the design matrix
   
Using the dictionary of starttimes (indicating when the window for data extraction should start), we extract a set of features. The features are usually first/last/min/max, and are defined in the utils subfunction (`vars_of_interest`).

In [ ]:
analyses = ['00', '04', '08', '16', '24', # 'base', 
            'fixed', 'wt24_fixed', 'wt8', 'wt16', 'wt24']
            #'wt8_00', 'wt8_08', 'wt8_16', 'wt8_24']

seeds = {'base': 897234762,
    'base_nodeathfix': 897234762,
    '00': 85134,
    '04': 64532,
    '08': 10928,
    '16': 98432,
    '24': 98382,
    'fixed': 48768,
    'wt8': 743762,
    'wt16': 81230,
    'wt24': 809172,
    'wt8_00': 45133,
    'wt8_08': 13749,
    'wt8_16': 85699,
    'wt8_24': 34651,
    'wt24_fixed': 19862}

for i, a in enumerate(analyses):
    # create a random offset for each patient's stay
    np.random.seed(seed=seeds[a])
    
    # set the window length
    if a in ['base','base_nodeathfix', 'fixed',
            '00', '04', '08', '16', '24']:
        T = 4
        
    elif a in ['wt8']:
        T = 8
        
    elif a in ['wt16']:
        T = 16
        
    elif a in ['wt24']:
        T = 24
    
    elif 'wt8' in a and len(a)>3:
        T = 8
        T_before_death = int(a[-2:])
        
    # determine if we should fix the time for those who die
    if a in ['00', '04', '08', '16', '24']:
        T_before_death = int(a)
    else:
        T_before_death = None
    
    
    # generate the start dictionary
    if a == 'base_nodeathfix':
        start_dict = mp.gen_random_offset(df_static, death_fix=False, T = T, T_before_death = T_before_death)
        
    elif 'fixed' in a:
        df_tmp = df_static.copy()
        df_tmp['zeros'] = 0
        start_dict = df_tmp[['icustay_id','zeros']].set_index('icustay_id').to_dict()['zeros']
        
    else:
        start_dict = mp.gen_random_offset(df_static, T = T, T_before_death = T_before_death)
        
        
    file_ext = '_' + a
    
    # write the static data - with the randomly generated offsets - to file
    df_static.set_index('icustay_id').to_csv('./icustays_offset' + file_ext + '.csv')

    print('Beginning prepping data for {}: {}.'.format(a,dt.datetime.now()))
    df_data = mp.extract_feature_ap(df, start_dict, offset = T*60.0)

    # write the new data to csv
    df_data.to_csv('design_matrix' + file_ext + '.csv',index_label='icustay_id')

    print('Finished prepping data for {}: {}.'.format(a,dt.datetime.now()))